In [1]:
import numpy as np

## Initialize Parameters for L Neural Network

In [3]:
def initialize_parameters(layer_dim, seed = 3):

    """
    Arguments:
    layer_dim: Array with the dimensions of each layer of the network
    seed: random seed
    
    Return:
    W: weight matrix (n_l, n_l-1)
    b: bias vector (n_l, 1)
    
    """

    np.random.seed(seed)
    
    parameters = {}
    
    L = len(layer_dim) # Layers in the network
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dim[l], layer_dim[l-1]) * 0.01
        parameters["b" + str(l)] = np.zeros((layer_dim[l], 1))

    return parameters

**This is an example for Neural Network**


<img src="Images/1.png" width="600">   

## Forward Propagation

<img src="Images/2.png" width="600">  

<img src="Images/3.png" width="600">  

In [9]:
def linear_forward(A, W, b):

    """
    Arguments
    A: Values form the previous layer (A0 is the input layer with X values)
    W: weight matrix
    b: bias vector
    
    Return
    Z: Preactivaction parameter
    cache: a python tuple containing "A", "W" and "b" ; stored for computing the backward pass
    
    """

    Z = np.dot(W, A) + b

    cache = (A, W, b)

    return Z, cache

In [10]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Arguments
    A_prev: activations from previous layer (or input data)
    W: weights matrix
    b: bias vector
    activation: the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A: the output of the activation function
    cache: a python tuple containing "linear_cache" and "activation_cache" stored for computing the backward pass
    """
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = (1/(1+np.exp(-Z)), Z)
        
    elif activation == "relu":                      
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = (np.maximum(0, Z), Z)
        
    cache = (linear_cache, activation_cache)

    return A, cache

In [11]:
def L_model_forward(X, parameters):
    """
    Arguments
    X: data
    parameters: output of initialize_parameters()
    
    Returns:
    AL: activation value from the output (last) layer
    caches: list of caches containing: every cache of linear_activation_forward() (there are L of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    # The for loop starts at 1 because layer 0 is the input
    for l in range(1, L):
        A_prev = A 
        W = parameters[f"W{l}"]
        b = parameters[f"b{l}"]
        A, cache = linear_activation_forward(A_prev, W, b, activation="relu")
        caches.append(cache)
        
    
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    W = parameters[f"W{L}"]
    b = parameters[f"b{L}"]
    AL, cache = linear_activation_forward(A, W, b, activation="sigmoid")
    caches.append(cache)
  
          
    return AL, caches

## Cost Function

<img src="Images/4.png" width="600">  

In [14]:
def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions
    Y -- true "label" vector 

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]
    
    cost = -1/m * np.sum(Y * np.log(AL) + (1 - Y) * np.log(1 - AL))
         
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).

    return cost

## Backwward Propagation

<img src="Images/5.png" width="600">  